# Pandas 데모 - Explarotary Data Analysis

## 라이브러리 import 및 설정

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import lightgbm as lgb
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import seaborn as sns
import warnings
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
import math
from sklearn.metrics import confusion_matrix

from sklearn.preprocessing import StandardScaler, RobustScaler # 데이터 표준화

# lightgbm 모델
from lightgbm import LGBMClassifier, plot_importance

# normalization
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

#feature Selection
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import f_classif
import itertools

In [4]:
rcParams['figure.figsize'] = (18, 9)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

## 데이터 다운로드

데이터는 [Dacon 단국대 소/중 데이터 분석 AI 경진대회 웹사이트](https://www.dacon.io/competitions/official/235638/data/)에서 다운로드 받아 `../input` 폴더에 저장.

In [5]:
!ls -alF ../input/

total 93780
drwxr-xr-x 2 swcu swcu     4096 Sep 28 05:26 ./
drwxr-xr-x 7 swcu swcu     4096 Sep 28 05:31 ../
-rw-rw-r-- 1 swcu swcu   800010 Jul 26 04:35 sample_submission.csv
-rw-rw-r-- 1 swcu swcu 18936246 Jul 26 04:35 test.csv
-rw-rw-r-- 1 swcu swcu 76278443 Jul 26 04:35 train.csv


In [6]:
data_dir = Path('../input/')
trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test.csv'
# feature_file = data_dir / 'feature.csv'
sample_file = data_dir / 'sample_submission.csv'
submission_dir = Path('../submission')
target_col = 'class'
seed = 42

In [7]:
algo_name = 'lgb'
feature_name = 'feature'
model_name = f'{algo_name}_{feature_name}'

#feature_file = feature_dir / f'{feature_name}.csv'
submission_file = submission_dir / f'{model_name}.csv'

## EDA

In [8]:
trn = pd.read_csv(trn_file, index_col=0)
print(trn.shape)
trn.head()

(320000, 19)


,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
id,,,,,,,,,,,,,,,,,,,
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19.5552,4,3,1.2054,1.2061,1.2049,1.2051,1.2057,0
4,24.4534,20.6992,19.0424,18.3242,17.9826,-3.3247e-05,23.7714,20.4338,18.8630,18.1903,17.8759,13,12,1.1939,1.1943,1.1937,1.1938,1.1941,0


## 데이터 개요

In [9]:
trn.describe()

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
count,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,3.2000e+05,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000,320000.0000
mean,19.8524,18.3768,17.7358,17.1742,16.8752,5.8308e-02,19.6289,18.2271,17.6154,17.1523,16.8693,6.3327,6.1388,1.1758,1.1765,1.1751,1.1754,1.1761,1.1163
std,1.9455,31.1108,1.4662,52.5615,57.1463,5.7546e-01,1.9236,17.7866,1.4730,25.0758,35.4365,8.8811,8.5516,0.1163,0.1181,0.1147,0.1155,0.1172,0.9235
min,-17.6875,-17555.3656,7.7314,-23955.8178,-23955.8178,-1.6605e+02,-42.8375,-9999.0000,-16.3802,-9999.0000,-9999.0000,1.0000,1.0000,1.0001,1.0000,1.0002,1.0001,1.0000,0.0000
25%,18.7242,17.4749,16.8772,16.5239,16.2889,3.5580e-05,18.5636,17.3484,16.7871,16.4530,16.2341,1.0000,1.0000,1.0882,1.0884,1.0878,1.0880,1.0883,0.0000
50%,19.4194,18.1405,17.5258,17.1497,16.9176,4.7129e-02,19.2646,18.0224,17.4434,17.0874,16.8694,2.0000,2.0000,1.1793,1.1792,1.1794,1.1794,1.1793,1.0000
75%,20.4321,19.0732,18.4281,18.0074,17.7289,9.4597e-02,20.1976,18.8884,18.2911,17.9070,17.6557,5.0000,5.0000,1.2275,1.2260,1.2292,1.2286,1.2268,2.0000
max,58.6233,46.3383,72.0097,50.1641,52.6127,6.2323e+01,30.7779,30.6132,31.9572,32.0162,30.7813,44.0000,42.0000,2.0508,2.0803,2.0221,2.0363,2.0654,2.0000


In [10]:
trn.dtypes

u            float64
g            float64
r            float64
i            float64
z            float64
redshift     float64
dered_u      float64
dered_g      float64
dered_r      float64
dered_i      float64
dered_z      float64
nObserve       int64
nDetect        int64
airmass_u    float64
airmass_g    float64
airmass_r    float64
airmass_i    float64
airmass_z    float64
class          int64
dtype: object

In [11]:
for i in trn.columns:
    print(i, len(trn.loc[trn[i]<0]))


u 3
g 4
r 0
i 4
z 9
redshift 70876
dered_u 8
dered_g 9
dered_r 6
dered_i 7
dered_z 8
nObserve 0
nDetect 0
airmass_u 0
airmass_g 0
airmass_r 0
airmass_i 0
airmass_z 0
class 0


In [12]:
col = ['u', 'g', 'r', 'i', 'z', 'dered_u', 'dered_g', 
      'dered_r', 'dered_i', 'dered_z']
trn1 = trn.copy()
for i in col:
    trn1 = trn1[trn1[i] >=0]

trn1

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
id,,,,,,,,,,,,,,,,,,,
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19.5552,4,3,1.2054,1.2061,1.2049,1.2051,1.2057,0
4,24.4534,20.6992,19.0424,18.3242,17.9826,-3.3247e-05,23.7714,20.4338,18.8630,18.1903,17.8759,13,12,1.1939,1.1943,1.1937,1.1938,1.1941,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319995,18.3656,17.2409,16.7739,16.4850,16.3404,5.1988e-02,18.1328,17.0406,16.6374,16.3861,16.2874,2,2,1.0930,1.0888,1.0973,1.0951,1.0909,2
319996,18.6856,17.6658,17.1904,16.8169,16.6367,8.7261e-02,18.5207,17.5131,17.0871,16.7436,16.6017,2,2,1.0517,1.0508,1.0526,1.0522,1.0513,1
319997,21.1890,18.4234,17.2956,16.9011,16.6841,-1.0148e-05,20.9890,18.3076,17.2152,16.8403,16.6355,31,30,1.1933,1.1941,1.1925,1.1929,1.1937,0


In [13]:
trn1.describe()

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
count,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,3.1998e+05,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000,319978.0000
mean,19.8528,18.4320,17.7358,17.3043,17.0613,5.8309e-02,19.6301,18.2593,17.6161,17.2154,16.9947,6.3328,6.1390,1.1758,1.1765,1.1751,1.1754,1.1761,1.1163
std,1.9428,1.6591,1.4662,1.3198,1.3333,5.7548e-01,1.9072,1.6566,1.4665,1.3249,1.3233,8.8814,8.5519,0.1163,0.1181,0.1147,0.1155,0.1172,0.9235
min,2.2651,1.2383,7.7314,2.7492,5.1281,-1.6605e+02,0.7538,3.0828,6.0403,6.8561,7.2567,1.0000,1.0000,1.0001,1.0000,1.0002,1.0001,1.0000,0.0000
25%,18.7242,17.4749,16.8772,16.5239,16.2890,3.5580e-05,18.5637,17.3484,16.7873,16.4531,16.2342,1.0000,1.0000,1.0882,1.0884,1.0878,1.0880,1.0883,0.0000
50%,19.4195,18.1405,17.5258,17.1498,16.9177,4.7133e-02,19.2646,18.0224,17.4434,17.0874,16.8694,2.0000,2.0000,1.1793,1.1792,1.1794,1.1794,1.1793,1.0000
75%,20.4320,19.0729,18.4281,18.0075,17.7290,9.4600e-02,20.1976,18.8885,18.2912,17.9070,17.6556,5.0000,5.0000,1.2275,1.2260,1.2292,1.2286,1.2268,2.0000
max,58.6233,46.3383,72.0097,50.1641,52.6127,6.2323e+01,30.7779,30.6132,31.9572,32.0162,30.7813,44.0000,42.0000,2.0508,2.0803,2.0221,2.0363,2.0654,2.0000


In [14]:
tst = pd.read_csv(tst_file, index_col = 0)
tst

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z
id,,,,,,,,,,,,,,,,,,
320000,19.1431,18.0172,17.4733,17.1144,16.9263,1.2130e-01,19.0750,17.9550,17.4308,17.0838,16.9087,1,1,1.1563,1.1515,1.1613,1.1588,1.1539
320001,18.7409,17.1705,16.3691,15.9865,15.7307,-6.6907e-05,15.0703,14.3098,14.3901,14.5160,14.6372,2,2,1.0076,1.0074,1.0080,1.0078,1.0075
320002,19.2980,17.8803,17.2221,16.8096,16.6352,1.1001e-01,19.1004,17.7726,17.1460,16.7497,16.5689,1,1,1.3809,1.3905,1.3715,1.3761,1.3856
320003,18.0775,17.1569,16.9041,16.8118,16.7955,2.9697e-05,17.9729,17.0735,16.8470,16.7700,16.7668,1,1,1.1408,1.1408,1.1409,1.1408,1.1408
320004,17.8325,16.7646,16.2135,15.8722,15.6621,6.8471e-02,17.7545,16.6794,16.1563,15.8315,15.6433,2,2,1.1110,1.1077,1.1145,1.1127,1.1093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,19.6734,18.6617,18.2708,18.1225,18.0627,-3.1726e-04,19.3361,18.3933,18.0860,17.9861,17.9670,17,17,1.1858,1.1863,1.1855,1.1857,1.1861
399996,21.5229,20.7283,20.6087,21.0049,20.8458,6.5455e-02,21.3993,20.6399,20.5451,20.9507,20.7827,8,4,1.2850,1.2908,1.2793,1.2821,1.2878
399997,19.3991,18.2411,18.1605,18.1978,18.2018,-8.4180e-04,18.9136,17.8753,17.9047,18.0043,18.0400,1,1,1.1885,1.1886,1.1885,1.1885,1.1885


In [15]:
df = pd.concat([trn1, tst])
df

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class
id,,,,,,,,,,,,,,,,,,,
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0.0
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1.0
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0.0
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19.5552,4,3,1.2054,1.2061,1.2049,1.2051,1.2057,0.0
4,24.4534,20.6992,19.0424,18.3242,17.9826,-3.3247e-05,23.7714,20.4338,18.8630,18.1903,17.8759,13,12,1.1939,1.1943,1.1937,1.1938,1.1941,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,19.6734,18.6617,18.2708,18.1225,18.0627,-3.1726e-04,19.3361,18.3933,18.0860,17.9861,17.9670,17,17,1.1858,1.1863,1.1855,1.1857,1.1861,NaN
399996,21.5229,20.7283,20.6087,21.0049,20.8458,6.5455e-02,21.3993,20.6399,20.5451,20.9507,20.7827,8,4,1.2850,1.2908,1.2793,1.2821,1.2878,NaN
399997,19.3991,18.2411,18.1605,18.1978,18.2018,-8.4180e-04,18.9136,17.8753,17.9047,18.0043,18.0400,1,1,1.1885,1.1886,1.1885,1.1885,1.1885,NaN


# Feature Engineering

In [16]:
target_col = 'class'
feature_col = [x for x in trn1.columns if trn1[x].dtype in [np.int64, np.float64] and x !=target_col]
len(feature_col)

18

In [17]:
oneChar_col = ['u','g','r','i','z']
dered_col = [c for c in df.columns if c.find('dered') != -1]
airmass_col = [c for c in df.columns if c.find('airmass') != -1]
print(dered_col)
print(airmass_col)

['dered_u', 'dered_g', 'dered_r', 'dered_i', 'dered_z']
['airmass_u', 'airmass_g', 'airmass_r', 'airmass_i', 'airmass_z']


In [18]:
#모든 조합 차이 feature
diff_feature = []
for c1, c2 in itertools.combinations(feature_col,2):
    new_c = f'{c1}_{c2}_diff'
    df[new_c] = df[c1] - df[c2]
    diff_feature.append(new_c)
# 모든 조합 나누기    
divide_feature = []
for c1, c2 in itertools.combinations(feature_col,2):
    new_c = f'{c1}_{c2}_divide'
    df[new_c] = df[c1] / df[c2]
    divide_feature.append(new_c)

df.columns.shape

(325,)

## Adding PCA 

In [19]:
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA
from sklearn.decomposition import KernelPCA



In [20]:
#default pca - 
temp = df.columns
temp = temp.drop('class')
pca = PCA(n_components=0.95)# 주성분 개수 설정 
X_pca = pca.fit_transform(df[temp])
print('eigen_value :', pca.explained_variance_)
print('explained variance ratio :', pca.explained_variance_ratio_)


df['PCA'] = X_pca
df['PCA'].shape

eigen_value : [1.11660542e+16]
explained variance ratio : [0.99948469]


(399978,)

In [21]:
X_pca.shape

(399978, 1)

In [22]:
# incrementalPCA - difference is just dividing n_batches for memory 
n_batches = 100
inc_pca = IncrementalPCA(n_components=18)#temp.size*0.5)
for batch_x in np.array_split(df[temp], n_batches):
    print(".", end="") # not shown in the book
    inc_pca.partial_fit(batch_x)
    
X_inc = inc_pca.transform(df[temp])


# print('eigen_value :', inc_pca.explained_variance_)
# print('explained variance ratio :', inc_pca.explained_variance_ratio_)

# X_inc

X_inc_t = X_inc.transpose()

....................................................................................................

In [23]:
for i , ele in enumerate(X_inc_t):
    incPCAn = 'incPCA'+str(i)
    df[incPCAn] = ele
df

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class,u_g_diff,u_r_diff,u_i_diff,u_z_diff,u_redshift_diff,u_dered_u_diff,u_dered_g_diff,u_dered_r_diff,u_dered_i_diff,u_dered_z_diff,u_nObserve_diff,u_nDetect_diff,u_airmass_u_diff,u_airmass_g_diff,u_airmass_r_diff,u_airmass_i_diff,u_airmass_z_diff,g_r_diff,g_i_diff,g_z_diff,g_redshift_diff,g_dered_u_diff,g_dered_g_diff,g_dered_r_diff,g_dered_i_diff,g_dered_z_diff,g_nObserve_diff,g_nDetect_diff,g_airmass_u_diff,g_airmass_g_diff,g_airmass_r_diff,...,dered_i_airmass_r_divide,dered_i_airmass_i_divide,dered_i_airmass_z_divide,dered_z_nObserve_divide,dered_z_nDetect_divide,dered_z_airmass_u_divide,dered_z_airmass_g_divide,dered_z_airmass_r_divide,dered_z_airmass_i_divide,dered_z_airmass_z_divide,nObserve_nDetect_divide,nObserve_airmass_u_divide,nObserve_airmass_g_divide,nObserve_airmass_r_divide,nObserve_airmass_i_divide,nObserve_airmass_z_divide,nDetect_airmass_u_divide,nDetect_airmass_g_divide,nDetect_airmass_r_divide,nDetect_airmass_i_divide,nDetect_airmass_z_divide,airmass_u_airmass_g_divide,airmass_u_airmass_r_divide,airmass_u_airmass_i_divide,airmass_u_airmass_z_divide,airmass_g_airmass_r_divide,airmass_g_airmass_i_divide,airmass_g_airmass_z_divide,airmass_r_airmass_i_divide,airmass_r_airmass_z_divide,airmass_i_airmass_z_divide,PCA,incPCA0,incPCA1,incPCA2,incPCA3,incPCA4,incPCA5,incPCA6,incPCA7,incPCA8,incPCA9,incPCA10,incPCA11,incPCA12,incPCA13,incPCA14,incPCA15,incPCA16,incPCA17
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0.0,2.9272,4.2545,5.5915,6.3243,23.2640,0.1397,3.0062,4.3088,5.6318,6.3550,5.2640,5.2640,22.0742,22.0733,22.0750,22.0746,22.0738,1.3273,2.6643,3.3972,20.3369,-2.7875,0.0790,1.3817,2.7047,3.4278,2.3368,2.3368,19.1470,19.1461,19.1478,...,14.8296,14.8249,14.8143,0.9394,0.9394,14.2120,14.2011,14.2214,14.2169,14.2067,1.0000,15.1291,15.1174,15.1390,15.1343,15.1234,15.1291,15.1174,15.1390,15.1343,15.1234,0.9992,1.0007,1.0003,0.9996,1.0014,1.0011,1.0004,0.9997,0.9990,0.9993,4.1012e+05,-4.1012e+05,-44588.1645,700.0365,-705.3298,-2043.2987,3.8336,-80.0839,-3.4657,17.4298,12.8590,6.3963,-1.8975,0.1039,-1.1943,-0.1168,-0.3289,-1.2392,0.8164
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1.0,0.9902,1.5997,1.7837,1.8832,15.0476,0.0857,1.0476,1.6408,1.8159,1.9175,14.0521,14.0521,13.7989,13.7943,13.8034,13.8012,13.7966,0.6096,0.7935,0.8930,14.0575,-0.9045,0.0574,0.6506,0.8257,0.9273,13.0620,13.0620,12.8087,12.8041,12.8132,...,10.5995,10.5806,10.5423,13.1347,13.1347,10.4805,10.4423,10.5182,10.4994,10.4614,1.0000,0.7979,0.7950,0.8008,0.7994,0.7965,0.7979,0.7950,0.8008,0.7994,0.7965,0.9964,1.0036,1.0018,0.9982,1.0073,1.0055,1.0018,0.9982,0.9946,0.9964,-1.3453e+05,1.3453e+05,1329.3993,234.1073,-139.3329,141.9778,0.8354,26.8378,4.3943,-49.9491,3.9271,6.1358,0.9277,-2.3727,0.1881,-0.6354,-0.4376,1.1678,-0.0957
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0.0,0.9610,1.2501,1.3928,1.4364,16.7859,0.1787,1.0998,1.3463,1.4647,1.4903,14.7864,14.7864,15.7639,15.7623,15.7654,15.7647,15.7631,0.2891,0.4319,0.4754,15.8250,-0.7822,0.1388,0.3854,0.5037,0.5293,13.8254,13.8254,14.8029,14.8013,14.8044,...,15.0068,14.9959,14.9730,7.6480,7.6480,14.9595,14.9360,14.9817,14.9708,14.9479,1.0000,1.9560,1.9529,1.9589,1.9575,1.9545,1.9560,1.9529,1.9589,1.9575,1.9545,0.9984,1.0015,1.0008,0.9992,1.0031,1.0023,1.0008,0.9993,0.9977,0.9985,-2.0247e+05,2.0247e+05,1399.9650,333.2776,117.0187,206.6743,-0.3252,15.6501,-15.6121,-13.6220,-9.5625,8.5752,0.3751,-0.9788,0.0770,-0.7371,0.0022,0.2375,0.3560
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,25.3536,20.9947,20.0873,

In [24]:
# from sklearn.model_selection import StratifiedShuffleSplit

# split = StratifiedShuffleSplit(n_splits=10, random_state=seed)

In [25]:
# trn2 = df.iloc[:319978]
# tst2 = df.iloc[319978:]
# tst2.drop('class', axis =1 , inplace = True)
# trn2.shape, tst2.shape

In [26]:
# trn2['class']

In [27]:
# for trn_idx, test_idx in split.split(trn2[temp], trn2['class']):
#     print (len(trn_idx), len(test_idx))
    
    

# testing = trn2[temp].iloc[test_idx]
    
# rbf_pca = KernelPCA(n_components = 100, kernel="rbf", gamma=0.0433)
# X_rbf = rbf_pca.fit_transform(testing)    

In [28]:
# #커널을 이용해 데이터를 저차원에서 고차원으로 매핑시켜 비선형 데이터셋에 SVM을 적용시키는 Kernel SVM, 
# #같은 기법을 PCA에 적용해 비선형 투영으로 차원을 축소할 수 있는데, 이것을 Kernel PCA(KPCA)라고 한다.



# rbf_pca = KernelPCA(n_components = 100, kernel="rbf", gamma=0.0433)
# X_rbf = rbf_pca.fit_transform(df[temp])

# print('eigen_value :', X_rbf.explained_variance_)
# print('explained variance ratio :', X_rbf.explained_variance_ratio_)
# print(X_rbf)

# lin_pca = KernelPCA(n_components = 0.95, kernel="linear") #fit_inverse_transform=True)
# X_lin = lin_pca.fit_transform(df[temp])

# print('eigen_value :', X_lin.explained_variance_)
# print('explained variance ratio :', X_lin.explained_variance_ratio_)
# print(X_lin)


# sig_pca = KernelPCA(n_components = 0.95, kernel="sigmoid", gamma=0.001, coef0=1)# fit_inverse_transform=True)
# X_sig = sig_pca.fit_transform(df[temp])

# print('eigen_value :', X_sig.explained_variance_)
# print('explained variance ratio :', X_sig.explained_variance_ratio_)
# print(X_sig)









In [29]:
df

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class,u_g_diff,u_r_diff,u_i_diff,u_z_diff,u_redshift_diff,u_dered_u_diff,u_dered_g_diff,u_dered_r_diff,u_dered_i_diff,u_dered_z_diff,u_nObserve_diff,u_nDetect_diff,u_airmass_u_diff,u_airmass_g_diff,u_airmass_r_diff,u_airmass_i_diff,u_airmass_z_diff,g_r_diff,g_i_diff,g_z_diff,g_redshift_diff,g_dered_u_diff,g_dered_g_diff,g_dered_r_diff,g_dered_i_diff,g_dered_z_diff,g_nObserve_diff,g_nDetect_diff,g_airmass_u_diff,g_airmass_g_diff,g_airmass_r_diff,...,dered_i_airmass_r_divide,dered_i_airmass_i_divide,dered_i_airmass_z_divide,dered_z_nObserve_divide,dered_z_nDetect_divide,dered_z_airmass_u_divide,dered_z_airmass_g_divide,dered_z_airmass_r_divide,dered_z_airmass_i_divide,dered_z_airmass_z_divide,nObserve_nDetect_divide,nObserve_airmass_u_divide,nObserve_airmass_g_divide,nObserve_airmass_r_divide,nObserve_airmass_i_divide,nObserve_airmass_z_divide,nDetect_airmass_u_divide,nDetect_airmass_g_divide,nDetect_airmass_r_divide,nDetect_airmass_i_divide,nDetect_airmass_z_divide,airmass_u_airmass_g_divide,airmass_u_airmass_r_divide,airmass_u_airmass_i_divide,airmass_u_airmass_z_divide,airmass_g_airmass_r_divide,airmass_g_airmass_i_divide,airmass_g_airmass_z_divide,airmass_r_airmass_i_divide,airmass_r_airmass_z_divide,airmass_i_airmass_z_divide,PCA,incPCA0,incPCA1,incPCA2,incPCA3,incPCA4,incPCA5,incPCA6,incPCA7,incPCA8,incPCA9,incPCA10,incPCA11,incPCA12,incPCA13,incPCA14,incPCA15,incPCA16,incPCA17
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0.0,2.9272,4.2545,5.5915,6.3243,23.2640,0.1397,3.0062,4.3088,5.6318,6.3550,5.2640,5.2640,22.0742,22.0733,22.0750,22.0746,22.0738,1.3273,2.6643,3.3972,20.3369,-2.7875,0.0790,1.3817,2.7047,3.4278,2.3368,2.3368,19.1470,19.1461,19.1478,...,14.8296,14.8249,14.8143,0.9394,0.9394,14.2120,14.2011,14.2214,14.2169,14.2067,1.0000,15.1291,15.1174,15.1390,15.1343,15.1234,15.1291,15.1174,15.1390,15.1343,15.1234,0.9992,1.0007,1.0003,0.9996,1.0014,1.0011,1.0004,0.9997,0.9990,0.9993,4.1012e+05,-4.1012e+05,-44588.1645,700.0365,-705.3298,-2043.2987,3.8336,-80.0839,-3.4657,17.4298,12.8590,6.3963,-1.8975,0.1039,-1.1943,-0.1168,-0.3289,-1.2392,0.8164
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1.0,0.9902,1.5997,1.7837,1.8832,15.0476,0.0857,1.0476,1.6408,1.8159,1.9175,14.0521,14.0521,13.7989,13.7943,13.8034,13.8012,13.7966,0.6096,0.7935,0.8930,14.0575,-0.9045,0.0574,0.6506,0.8257,0.9273,13.0620,13.0620,12.8087,12.8041,12.8132,...,10.5995,10.5806,10.5423,13.1347,13.1347,10.4805,10.4423,10.5182,10.4994,10.4614,1.0000,0.7979,0.7950,0.8008,0.7994,0.7965,0.7979,0.7950,0.8008,0.7994,0.7965,0.9964,1.0036,1.0018,0.9982,1.0073,1.0055,1.0018,0.9982,0.9946,0.9964,-1.3453e+05,1.3453e+05,1329.3993,234.1073,-139.3329,141.9778,0.8354,26.8378,4.3943,-49.9491,3.9271,6.1358,0.9277,-2.3727,0.1881,-0.6354,-0.4376,1.1678,-0.0957
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0.0,0.9610,1.2501,1.3928,1.4364,16.7859,0.1787,1.0998,1.3463,1.4647,1.4903,14.7864,14.7864,15.7639,15.7623,15.7654,15.7647,15.7631,0.2891,0.4319,0.4754,15.8250,-0.7822,0.1388,0.3854,0.5037,0.5293,13.8254,13.8254,14.8029,14.8013,14.8044,...,15.0068,14.9959,14.9730,7.6480,7.6480,14.9595,14.9360,14.9817,14.9708,14.9479,1.0000,1.9560,1.9529,1.9589,1.9575,1.9545,1.9560,1.9529,1.9589,1.9575,1.9545,0.9984,1.0015,1.0008,0.9992,1.0031,1.0023,1.0008,0.9993,0.9977,0.9985,-2.0247e+05,2.0247e+05,1399.9650,333.2776,117.0187,206.6743,-0.3252,15.6501,-15.6121,-13.6220,-9.5625,8.5752,0.3751,-0.9788,0.0770,-0.7371,0.0022,0.2375,0.3560
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,25.3536,20.9947,20.0873,

In [30]:
# zip 함수를 이용한 row별, 시리즈 그룹별 max, min, max-min, std, sum 을 feature로 생성
# 그 외에도 mean, skew 등을 사용해 볼 수도 있긴 함 

for prefix, g in zip(['one','dered','airmass'], [oneChar_col, dered_col, airmass_col]):
    
    df[f'{prefix}_max'] = df[g].max(axis=1)
    
    df[f'{prefix}_min'] = df[g].min(axis=1)
    
    df[f'{prefix}_diff'] = df[f'{prefix}_max'] - df[f'{prefix}_min']
    
    df[f'{prefix}_std'] = df[g].std(axis=1)
    
    df[f'{prefix}_sum'] = df[g].sum(axis=1)
    
    

In [31]:
# 각 그룹별 max-max, min-min, sum-sum, std-std을 feature로 생성

for c in itertools.combinations(['one','dered','airmass'],2):
    df[f'{c[0]}_{c[1]}_max_diff'] = df[f'{c[0]}_max'] - df[f'{c[1]}_max']
    
    df[f'{c[0]}_{c[1]}_min_diff'] = df[f'{c[0]}_min'] - df[f'{c[1]}_min']
    
    df[f'{c[0]}_{c[1]}_sum_diff'] = df[f'{c[0]}_sum'] - df[f'{c[1]}_sum']
    
    df[f'{c[0]}_{c[1]}_std_diff'] = df[f'{c[0]}_std'] - df[f'{c[1]}_std']

In [32]:
df

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class,u_g_diff,u_r_diff,u_i_diff,u_z_diff,u_redshift_diff,u_dered_u_diff,u_dered_g_diff,u_dered_r_diff,u_dered_i_diff,u_dered_z_diff,u_nObserve_diff,u_nDetect_diff,u_airmass_u_diff,u_airmass_g_diff,u_airmass_r_diff,u_airmass_i_diff,u_airmass_z_diff,g_r_diff,g_i_diff,g_z_diff,g_redshift_diff,g_dered_u_diff,g_dered_g_diff,g_dered_r_diff,g_dered_i_diff,g_dered_z_diff,g_nObserve_diff,g_nDetect_diff,g_airmass_u_diff,g_airmass_g_diff,g_airmass_r_diff,...,airmass_g_airmass_z_divide,airmass_r_airmass_i_divide,airmass_r_airmass_z_divide,airmass_i_airmass_z_divide,PCA,incPCA0,incPCA1,incPCA2,incPCA3,incPCA4,incPCA5,incPCA6,incPCA7,incPCA8,incPCA9,incPCA10,incPCA11,incPCA12,incPCA13,incPCA14,incPCA15,incPCA16,incPCA17,one_max,one_min,one_diff,one_std,one_sum,dered_max,dered_min,dered_diff,dered_std,dered_sum,airmass_max,airmass_min,airmass_diff,airmass_std,airmass_sum,one_dered_max_diff,one_dered_min_diff,one_dered_sum_diff,one_dered_std_diff,one_airmass_max_diff,one_airmass_min_diff,one_airmass_sum_diff,one_airmass_std_diff,dered_airmass_max_diff,dered_airmass_min_diff,dered_airmass_sum_diff,dered_airmass_std_diff
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0.0,2.9272,4.2545,5.5915,6.3243,23.2640,0.1397,3.0062,4.3088,5.6318,6.3550,5.2640,5.2640,22.0742,22.0733,22.0750,22.0746,22.0738,1.3273,2.6643,3.3972,20.3369,-2.7875,0.0790,1.3817,2.7047,3.4278,2.3368,2.3368,19.1470,19.1461,19.1478,...,1.0004,0.9997,0.9990,0.9993,4.1012e+05,-4.1012e+05,-44588.1645,700.0365,-705.3298,-2043.2987,3.8336,-80.0839,-3.4657,17.4298,12.8590,6.3963,-1.8975,0.1039,-1.1943,-0.1168,-0.3289,-1.2392,0.8164,23.2640,16.9396,6.3243,2.4994,97.2223,23.1243,16.9089,6.2153,2.4562,96.8782,1.1907,1.1890,1.7020e-03,6.7348e-04,5.9490,0.1397,0.0307,0.3441,0.0432,22.0733,15.7506,91.2733,2.4987,21.9336,15.7200,90.9292,2.4555
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1.0,0.9902,1.5997,1.7837,1.8832,15.0476,0.0857,1.0476,1.6408,1.8159,1.9175,14.0521,14.0521,13.7989,13.7943,13.8034,13.8012,13.7966,0.6096,0.7935,0.8930,14.0575,-0.9045,0.0574,0.6506,0.8257,0.9273,13.0620,13.0620,12.8087,12.8041,12.8132,...,1.0018,0.9982,0.9946,0.9964,-1.3453e+05,1.3453e+05,1329.3993,234.1073,-139.3329,141.9778,0.8354,26.8378,4.3943,-49.9491,3.9271,6.1358,0.9277,-2.3727,0.1881,-0.6354,-0.4376,1.1678,-0.0957,15.0521,13.1689,1.8832,0.7807,69.0039,14.9664,13.1347,1.8317,0.7586,68.7533,1.2578,1.2488,9.0740e-03,3.5867e-03,6.2664,0.0857,0.0343,0.2507,0.0222,13.7943,11.9202,62.7375,0.7771,13.7086,11.8859,62.4869,0.7550
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0.0,0.9610,1.2501,1.3928,1.4364,16.7859,0.1787,1.0998,1.3463,1.4647,1.4903,14.7864,14.7864,15.7639,15.7623,15.7654,15.7647,15.7631,0.2891,0.4319,0.4754,15.8250,-0.7822,0.1388,0.3854,0.5037,0.5293,13.8254,13.8254,14.8029,14.8013,14.8044,...,1.0008,0.9993,0.9977,0.9985,-2.0247e+05,2.0247e+05,1399.9650,333.2776,117.0187,206.6743,-0.3252,15.6501,-15.6121,-13.6220,-9.5625,8.5752,0.3751,-0.9788,0.0770,-0.7371,0.0022,0.2375,0.3560,16.7864,15.3500,1.4364,0.5934,78.8917,16.6076,15.2961,1.3116,0.5462,78.3521,1.0241,1.0210,3.1220e-03,1.2343e-03,5.1126,0.1787,0.0540,0.5396,0.0472,15.7623,14.3290,73.7791,0.5922,15.5835,14.2751,73.2395,0.5450
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19.5552,4,3,1.2054,1.2061,1.2049,1.2051,1.2057,0.0,4.4719,5.4395,5.7657,6.0260,25.6606,0.3070,4.6660,5.5734,5.8660,6.1055,21.6606,22.6606,24.4552,24.4546,24.4558,24.4555,24.4549,0.9676,1.2938,1.5541,21.1887,-4.1649,0.1941,1.1015,1.3941,1.6335,17.1887,

# 훈련셋과 테스트셋 나누기

In [33]:
trn2 = df.iloc[:319978]
tst2 = df.iloc[319978:]
tst2.drop('class', axis =1 , inplace = True)
trn2.shape, tst2.shape

((319978, 371), (80000, 370))

# outlier

In [34]:
# 전체 데이터가 아닌 각 클래스별(0,1,2)로 이상치 처리
# Train_copy1 = Train_set.copy()

def remove_outlier(data, column):
    
    print(column)
    # 클래스 0처리
    column_data_0 = data[data['class']==0][column]
    quan_25_0 = np.percentile(column_data_0.values, 0.1) # 1분위수
    quan_75_0 = np.percentile(column_data_0.values, 99.9) # 3분위수
    iqr_0 = quan_75_0 - quan_25_0 # IQR 계산
    iqr_0 *= 1.5
    lowest_0 = quan_25_0 - iqr_0 # 최솟값 설정
    highest_0 = quan_75_0 + iqr_0 # 최댓값 설정
    outlier_index_0 = column_data_0[(column_data_0 < lowest_0) | (column_data_0 > highest_0)].index # 최솟값 ~ 최댓값 사이의 값만 출력
    print(len(outlier_index_0))
    
    # 클래스 1처리
    column_data_1 = data[data['class']==1][column]
    quan_25_1 = np.percentile(column_data_1.values, 0.1) # 1분위수
    quan_75_1 = np.percentile(column_data_1.values, 99.9) # 3분위수
    iqr_1 = quan_75_1 - quan_25_1 # IQR 계산
    iqr_1 *= 1.5
    lowest_1 = quan_25_1 - iqr_1 # 최솟값 설정
    highest_1 = quan_75_1 + iqr_1 # 최댓값 설정
    outlier_index_1 = column_data_1[(column_data_1 < lowest_1 )| (column_data_1 > highest_1)].index # 최솟값 ~ 최댓값 사이의 값만 출력
    print(len(outlier_index_1))
    
    # 클래스 2 처리
    column_data_2 = data[data['class']==2][column]
    quan_25_2 = np.percentile(column_data_2.values, 0.1) # 1분위수
    quan_75_2 = np.percentile(column_data_2.values, 99.9) # 3분위수
    iqr_2 = quan_75_2 - quan_25_2 # IQR 계산
    iqr_2 *= 1.5
    lowest_2 = quan_25_2 - iqr_2 # 최솟값 설정
    highest_2 = quan_75_2 + iqr_2 # 최댓값 설정
    outlier_index_2 = column_data_2[(column_data_2 < lowest_2 )| (column_data_2 > highest_2)].index # 최솟값 ~ 최댓값 사이의 값만 출력
    print(len(outlier_index_2))
    # outlier 행 제거
    data.drop(list(set(list(outlier_index_0) + list(outlier_index_1) + list(outlier_index_2))), axis = 0, inplace = True)
    print(data.shape)
    return data

In [35]:
col = list(trn2.columns)
col.remove('redshift')
col.remove('class')
# col

['u',
 'g',
 'r',
 'i',
 'z',
 'dered_u',
 'dered_g',
 'dered_r',
 'dered_i',
 'dered_z',
 'nObserve',
 'nDetect',
 'airmass_u',
 'airmass_g',
 'airmass_r',
 'airmass_i',
 'airmass_z',
 'u_g_diff',
 'u_r_diff',
 'u_i_diff',
 'u_z_diff',
 'u_redshift_diff',
 'u_dered_u_diff',
 'u_dered_g_diff',
 'u_dered_r_diff',
 'u_dered_i_diff',
 'u_dered_z_diff',
 'u_nObserve_diff',
 'u_nDetect_diff',
 'u_airmass_u_diff',
 'u_airmass_g_diff',
 'u_airmass_r_diff',
 'u_airmass_i_diff',
 'u_airmass_z_diff',
 'g_r_diff',
 'g_i_diff',
 'g_z_diff',
 'g_redshift_diff',
 'g_dered_u_diff',
 'g_dered_g_diff',
 'g_dered_r_diff',
 'g_dered_i_diff',
 'g_dered_z_diff',
 'g_nObserve_diff',
 'g_nDetect_diff',
 'g_airmass_u_diff',
 'g_airmass_g_diff',
 'g_airmass_r_diff',
 'g_airmass_i_diff',
 'g_airmass_z_diff',
 'r_i_diff',
 'r_z_diff',
 'r_redshift_diff',
 'r_dered_u_diff',
 'r_dered_g_diff',
 'r_dered_r_diff',
 'r_dered_i_diff',
 'r_dered_z_diff',
 'r_nObserve_diff',
 'r_nDetect_diff',
 'r_airmass_u_diff',
 'r_a

In [36]:
df2 = trn2.copy()
for i in col:
    
    ROT = remove_outlier(df2, i)
    
    df2 = ROT

df2

u
5
1
2
(319970, 371)
g
0
0
0
(319970, 371)
r
1
0
2
(319967, 371)
i
1
1
0
(319965, 371)
z
1
1
0
(319963, 371)
dered_u
0
0
0
(319963, 371)
dered_g
0
0
0
(319963, 371)
dered_r
0
0
0
(319963, 371)
dered_i
0
0
0
(319963, 371)
dered_z
0
0
0
(319963, 371)
nObserve
0
0
0
(319963, 371)
nDetect
0
0
0
(319963, 371)
airmass_u
0
0
0
(319963, 371)
airmass_g
0
0
0
(319963, 371)
airmass_r
0
0
0
(319963, 371)
airmass_i
0
0
0
(319963, 371)
airmass_z
0
0
0
(319963, 371)
u_g_diff
1
4
7
(319951, 371)
u_r_diff
0
1
4
(319946, 371)
u_i_diff
0
0
1
(319945, 371)
u_z_diff
0
1
1
(319943, 371)
u_redshift_diff
24
0
0
(319919, 371)
u_dered_u_diff
1
8
18
(319892, 371)
u_dered_g_diff
0
0
0
(319892, 371)
u_dered_r_diff
0
0
0
(319892, 371)
u_dered_i_diff
0
0
0
(319892, 371)
u_dered_z_diff
0
0
0
(319892, 371)
u_nObserve_diff
0
0
0
(319892, 371)
u_nDetect_diff
0
0
0
(319892, 371)
u_airmass_u_diff
0
0
0
(319892, 371)
u_airmass_g_diff
0
0
0
(319892, 371)
u_airmass_r_diff
0
0
0
(319892, 371)
u_airmass_i_diff
0
0
0
(319892, 

0
0
(319150, 371)
i_z_divide
1
0
3
(319146, 371)
i_redshift_divide
1
0
0
(319145, 371)
i_dered_u_divide
0
0
0
(319145, 371)
i_dered_g_divide
0
0
0
(319145, 371)
i_dered_r_divide
0
4
0
(319141, 371)
i_dered_i_divide
0
0
0
(319141, 371)
i_dered_z_divide
0
0
0
(319141, 371)
i_nObserve_divide
0
0
0
(319141, 371)
i_nDetect_divide
0
0
0
(319141, 371)
i_airmass_u_divide
0
0
0
(319141, 371)
i_airmass_g_divide
0
0
0
(319141, 371)
i_airmass_r_divide
0
0
0
(319141, 371)
i_airmass_i_divide
0
0
0
(319141, 371)
i_airmass_z_divide
0
0
0
(319141, 371)
z_redshift_divide
0
1
1
(319139, 371)
z_dered_u_divide
0
0
0
(319139, 371)
z_dered_g_divide
2
0
0
(319137, 371)
z_dered_r_divide
0
4
0
(319133, 371)
z_dered_i_divide
0
4
5
(319124, 371)
z_dered_z_divide
0
1
0
(319123, 371)
z_nObserve_divide
0
0
0
(319123, 371)
z_nDetect_divide
0
0
0
(319123, 371)
z_airmass_u_divide
0
0
0
(319123, 371)
z_airmass_g_divide
0
0
0
(319123, 371)
z_airmass_r_divide
0
0
0
(319123, 371)
z_airmass_i_divide
0
0
0
(319123, 371)
z_ai

,u,g,r,i,z,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,nDetect,airmass_u,airmass_g,airmass_r,airmass_i,airmass_z,class,u_g_diff,u_r_diff,u_i_diff,u_z_diff,u_redshift_diff,u_dered_u_diff,u_dered_g_diff,u_dered_r_diff,u_dered_i_diff,u_dered_z_diff,u_nObserve_diff,u_nDetect_diff,u_airmass_u_diff,u_airmass_g_diff,u_airmass_r_diff,u_airmass_i_diff,u_airmass_z_diff,g_r_diff,g_i_diff,g_z_diff,g_redshift_diff,g_dered_u_diff,g_dered_g_diff,g_dered_r_diff,g_dered_i_diff,g_dered_z_diff,g_nObserve_diff,g_nDetect_diff,g_airmass_u_diff,g_airmass_g_diff,g_airmass_r_diff,...,airmass_g_airmass_z_divide,airmass_r_airmass_i_divide,airmass_r_airmass_z_divide,airmass_i_airmass_z_divide,PCA,incPCA0,incPCA1,incPCA2,incPCA3,incPCA4,incPCA5,incPCA6,incPCA7,incPCA8,incPCA9,incPCA10,incPCA11,incPCA12,incPCA13,incPCA14,incPCA15,incPCA16,incPCA17,one_max,one_min,one_diff,one_std,one_sum,dered_max,dered_min,dered_diff,dered_std,dered_sum,airmass_max,airmass_min,airmass_diff,airmass_std,airmass_sum,one_dered_max_diff,one_dered_min_diff,one_dered_sum_diff,one_dered_std_diff,one_airmass_max_diff,one_airmass_min_diff,one_airmass_sum_diff,one_airmass_std_diff,dered_airmass_max_diff,dered_airmass_min_diff,dered_airmass_sum_diff,dered_airmass_std_diff
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,18,18,1.1898,1.1907,1.1890,1.1894,1.1902,0.0,2.9272,4.2545,5.5915,6.3243,23.2640,0.1397,3.0062,4.3088,5.6318,6.3550,5.2640,5.2640,22.0742,22.0733,22.0750,22.0746,22.0738,1.3273,2.6643,3.3972,20.3369,-2.7875,0.0790,1.3817,2.7047,3.4278,2.3368,2.3368,19.1470,19.1461,19.1478,...,1.0004,0.9997,0.9990,0.9993,4.1012e+05,-4.1012e+05,-44588.1645,700.0365,-705.3298,-2043.2987,3.8336,-80.0839,-3.4657,17.4298,12.8590,6.3963,-1.8975,0.1039,-1.1943,-0.1168,-0.3289,-1.2392,0.8164,23.2640,16.9396,6.3243,2.4994,97.2223,23.1243,16.9089,6.2153,2.4562,96.8782,1.1907,1.1890,0.0017,6.7348e-04,5.9490,0.1397,0.0307,0.3441,0.0432,22.0733,15.7506,91.2733,2.4987,21.9336,15.7200,90.9292,2.4555
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,1,1,1.2533,1.2578,1.2488,1.2510,1.2555,1.0,0.9902,1.5997,1.7837,1.8832,15.0476,0.0857,1.0476,1.6408,1.8159,1.9175,14.0521,14.0521,13.7989,13.7943,13.8034,13.8012,13.7966,0.6096,0.7935,0.8930,14.0575,-0.9045,0.0574,0.6506,0.8257,0.9273,13.0620,13.0620,12.8087,12.8041,12.8132,...,1.0018,0.9982,0.9946,0.9964,-1.3453e+05,1.3453e+05,1329.3993,234.1073,-139.3329,141.9778,0.8354,26.8378,4.3943,-49.9491,3.9271,6.1358,0.9277,-2.3727,0.1881,-0.6354,-0.4376,1.1678,-0.0957,15.0521,13.1689,1.8832,0.7807,69.0039,14.9664,13.1347,1.8317,0.7586,68.7533,1.2578,1.2488,0.0091,3.5867e-03,6.2664,0.0857,0.0343,0.2507,0.0222,13.7943,11.9202,62.7375,0.7771,13.7086,11.8859,62.4869,0.7550
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,2,2,1.0225,1.0241,1.0210,1.0217,1.0233,0.0,0.9610,1.2501,1.3928,1.4364,16.7859,0.1787,1.0998,1.3463,1.4647,1.4903,14.7864,14.7864,15.7639,15.7623,15.7654,15.7647,15.7631,0.2891,0.4319,0.4754,15.8250,-0.7822,0.1388,0.3854,0.5037,0.5293,13.8254,13.8254,14.8029,14.8013,14.8044,...,1.0008,0.9993,0.9977,0.9985,-2.0247e+05,2.0247e+05,1399.9650,333.2776,117.0187,206.6743,-0.3252,15.6501,-15.6121,-13.6220,-9.5625,8.5752,0.3751,-0.9788,0.0770,-0.7371,0.0022,0.2375,0.3560,16.7864,15.3500,1.4364,0.5934,78.8917,16.6076,15.2961,1.3116,0.5462,78.3521,1.0241,1.0210,0.0031,1.2343e-03,5.1126,0.1787,0.0540,0.5396,0.0472,15.7623,14.3290,73.7791,0.5922,15.5835,14.2751,73.2395,0.5450
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19.5552,4,3,1.2054,1.2061,1.2049,1.2051,1.2057,0.0,4.4719,5.4395,5.7657,6.0260,25.6606,0.3070,4.6660,5.5734,5.8660,6.1055,21.6606,22.6606,24.4552,24.4546,24.4558,24.4555,24.4549,0.9676,1.2938,1.5541,21.1887,-4.1649,0.1941,1.1015,1.3941,1.6335,17.1887,18.1887,19.9

In [37]:
target_col = 'class'
feature_col = [x for x in df.columns if df[x].dtype in [np.int64, np.float64] and x !=target_col]
print(feature_col)
pred_col='pred'


['u', 'g', 'r', 'i', 'z', 'redshift', 'dered_u', 'dered_g', 'dered_r', 'dered_i', 'dered_z', 'nObserve', 'nDetect', 'airmass_u', 'airmass_g', 'airmass_r', 'airmass_i', 'airmass_z', 'u_g_diff', 'u_r_diff', 'u_i_diff', 'u_z_diff', 'u_redshift_diff', 'u_dered_u_diff', 'u_dered_g_diff', 'u_dered_r_diff', 'u_dered_i_diff', 'u_dered_z_diff', 'u_nObserve_diff', 'u_nDetect_diff', 'u_airmass_u_diff', 'u_airmass_g_diff', 'u_airmass_r_diff', 'u_airmass_i_diff', 'u_airmass_z_diff', 'g_r_diff', 'g_i_diff', 'g_z_diff', 'g_redshift_diff', 'g_dered_u_diff', 'g_dered_g_diff', 'g_dered_r_diff', 'g_dered_i_diff', 'g_dered_z_diff', 'g_nObserve_diff', 'g_nDetect_diff', 'g_airmass_u_diff', 'g_airmass_g_diff', 'g_airmass_r_diff', 'g_airmass_i_diff', 'g_airmass_z_diff', 'r_i_diff', 'r_z_diff', 'r_redshift_diff', 'r_dered_u_diff', 'r_dered_g_diff', 'r_dered_r_diff', 'r_dered_i_diff', 'r_dered_z_diff', 'r_nObserve_diff', 'r_nDetect_diff', 'r_airmass_u_diff', 'r_airmass_g_diff', 'r_airmass_r_diff', 'r_airmass_i_

In [41]:
from sklearn.feature_selection import SelectFromModel
#LGBM 모델을 기반으로 속성들 선택
select = SelectFromModel(LGBMClassifier(max_depth=27, learning_rate = 0.005, n_estimators = 1000, boosting_type='gbrt',
                                       ))#, threshold = 0.3)
select.fit(df2[feature_col], df2[target_col])

train_sel = select.transform(df2[feature_col])

test_sel = select.transform(tst2[feature_col])

[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves


In [42]:
# SelectFromModel?

## 모델링

In [43]:
df2.shape

(318740, 371)

In [44]:
train_sel.shape

(318740, 90)

In [45]:
y = df2[target_col]
df2.drop(target_col, axis=1, inplace=True)

feature_name = df2.columns.tolist()
print(y.shape, df2.shape, tst2.shape)

(318740,) (318740, 370) (80000, 370)


In [46]:
X_trn, X_val, y_trn, y_val = train_test_split(train_sel, y, test_size=.01, random_state=seed)
print(X_trn.shape, X_val.shape, y_trn.shape, y_val.shape)

clf = lgb.LGBMClassifier(objective='multiclass',
                         n_estimators=12000,
                         num_leaves=512,
                         learning_rate=0.001,
                         min_child_samples=10,
                         subsample=.5,
                         subsample_freq=1,
                         colsample_bytree=.8,
                         random_state=seed,
                         max_depth = 27,
                         n_jobs=-1)
                        #boosting_type = 'goss')
clf.fit(X_trn, y_trn,
        eval_set=[(X_val, y_val)],
        eval_metric='multiclass',
        early_stopping_rounds=100)

p_val = clf.predict(X_val)


print(f'{accuracy_score(y_val, p_val) * 100:.4f}%')

(315552, 90) (3188, 90) (315552,) (3188,)
[1]	valid_0's multi_logloss: 0.990721
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_logloss: 0.989141
[3]	valid_0's multi_logloss: 0.987557
[4]	valid_0's multi_logloss: 0.985981
[5]	valid_0's multi_logloss: 0.984422
[6]	valid_0's multi_logloss: 0.982854
[7]	valid_0's multi_logloss: 0.981284
[8]	valid_0's multi_logloss: 0.979731
[9]	valid_0's multi_logloss: 0.978187
[10]	valid_0's multi_logloss: 0.976643
[11]	valid_0's multi_logloss: 0.97511
[12]	valid_0's multi_logloss: 0.973568
[13]	valid_0's multi_logloss: 0.972027
[14]	valid_0's multi_logloss: 0.970493
[15]	valid_0's multi_logloss: 0.968962
[16]	valid_0's multi_logloss: 0.96743
[17]	valid_0's multi_logloss: 0.965911
[18]	valid_0's multi_logloss: 0.964393
[19]	valid_0's multi_logloss: 0.96289
[20]	valid_0's multi_logloss: 0.961392
[21]	valid_0's multi_logloss: 0.959884
[22]	valid_0's multi_logloss: 0.958379
[23]	valid_0's multi_logloss: 0.956885
[24]	valid_

[207]	valid_0's multi_logloss: 0.737042
[208]	valid_0's multi_logloss: 0.736085
[209]	valid_0's multi_logloss: 0.735124
[210]	valid_0's multi_logloss: 0.734166
[211]	valid_0's multi_logloss: 0.733212
[212]	valid_0's multi_logloss: 0.732267
[213]	valid_0's multi_logloss: 0.731324
[214]	valid_0's multi_logloss: 0.730375
[215]	valid_0's multi_logloss: 0.729429
[216]	valid_0's multi_logloss: 0.728481
[217]	valid_0's multi_logloss: 0.727538
[218]	valid_0's multi_logloss: 0.726603
[219]	valid_0's multi_logloss: 0.725668
[220]	valid_0's multi_logloss: 0.724735
[221]	valid_0's multi_logloss: 0.723802
[222]	valid_0's multi_logloss: 0.72287
[223]	valid_0's multi_logloss: 0.721937
[224]	valid_0's multi_logloss: 0.721009
[225]	valid_0's multi_logloss: 0.72008
[226]	valid_0's multi_logloss: 0.719151
[227]	valid_0's multi_logloss: 0.718227
[228]	valid_0's multi_logloss: 0.717307
[229]	valid_0's multi_logloss: 0.71638
[230]	valid_0's multi_logloss: 0.715471
[231]	valid_0's multi_logloss: 0.714556
[23

[413]	valid_0's multi_logloss: 0.574898
[414]	valid_0's multi_logloss: 0.574252
[415]	valid_0's multi_logloss: 0.573605
[416]	valid_0's multi_logloss: 0.572967
[417]	valid_0's multi_logloss: 0.572329
[418]	valid_0's multi_logloss: 0.57169
[419]	valid_0's multi_logloss: 0.571049
[420]	valid_0's multi_logloss: 0.570417
[421]	valid_0's multi_logloss: 0.56979
[422]	valid_0's multi_logloss: 0.569159
[423]	valid_0's multi_logloss: 0.56853
[424]	valid_0's multi_logloss: 0.567902
[425]	valid_0's multi_logloss: 0.567273
[426]	valid_0's multi_logloss: 0.566641
[427]	valid_0's multi_logloss: 0.566011
[428]	valid_0's multi_logloss: 0.565381
[429]	valid_0's multi_logloss: 0.564755
[430]	valid_0's multi_logloss: 0.564129
[431]	valid_0's multi_logloss: 0.563501
[432]	valid_0's multi_logloss: 0.562875
[433]	valid_0's multi_logloss: 0.562264
[434]	valid_0's multi_logloss: 0.561643
[435]	valid_0's multi_logloss: 0.561023
[436]	valid_0's multi_logloss: 0.560405
[437]	valid_0's multi_logloss: 0.559788
[43

[620]	valid_0's multi_logloss: 0.463425
[621]	valid_0's multi_logloss: 0.462975
[622]	valid_0's multi_logloss: 0.462525
[623]	valid_0's multi_logloss: 0.462075
[624]	valid_0's multi_logloss: 0.461623
[625]	valid_0's multi_logloss: 0.461176
[626]	valid_0's multi_logloss: 0.46072
[627]	valid_0's multi_logloss: 0.460279
[628]	valid_0's multi_logloss: 0.459838
[629]	valid_0's multi_logloss: 0.4594
[630]	valid_0's multi_logloss: 0.458959
[631]	valid_0's multi_logloss: 0.458514
[632]	valid_0's multi_logloss: 0.458076
[633]	valid_0's multi_logloss: 0.457635
[634]	valid_0's multi_logloss: 0.457198
[635]	valid_0's multi_logloss: 0.45676
[636]	valid_0's multi_logloss: 0.456318
[637]	valid_0's multi_logloss: 0.455882
[638]	valid_0's multi_logloss: 0.455447
[639]	valid_0's multi_logloss: 0.455013
[640]	valid_0's multi_logloss: 0.454579
[641]	valid_0's multi_logloss: 0.454142
[642]	valid_0's multi_logloss: 0.45371
[643]	valid_0's multi_logloss: 0.453278
[644]	valid_0's multi_logloss: 0.452847
[645]

[826]	valid_0's multi_logloss: 0.384759
[827]	valid_0's multi_logloss: 0.384431
[828]	valid_0's multi_logloss: 0.384112
[829]	valid_0's multi_logloss: 0.383795
[830]	valid_0's multi_logloss: 0.383478
[831]	valid_0's multi_logloss: 0.383156
[832]	valid_0's multi_logloss: 0.382835
[833]	valid_0's multi_logloss: 0.382515
[834]	valid_0's multi_logloss: 0.382194
[835]	valid_0's multi_logloss: 0.381876
[836]	valid_0's multi_logloss: 0.38156
[837]	valid_0's multi_logloss: 0.381243
[838]	valid_0's multi_logloss: 0.380928
[839]	valid_0's multi_logloss: 0.380612
[840]	valid_0's multi_logloss: 0.380296
[841]	valid_0's multi_logloss: 0.37998
[842]	valid_0's multi_logloss: 0.379662
[843]	valid_0's multi_logloss: 0.379348
[844]	valid_0's multi_logloss: 0.379031
[845]	valid_0's multi_logloss: 0.378718
[846]	valid_0's multi_logloss: 0.378404
[847]	valid_0's multi_logloss: 0.378086
[848]	valid_0's multi_logloss: 0.37777
[849]	valid_0's multi_logloss: 0.377463
[850]	valid_0's multi_logloss: 0.37715
[851

...

[6472]	valid_0's multi_logloss: 0.146144
[6473]	valid_0's multi_logloss: 0.146142
[6474]	valid_0's multi_logloss: 0.146143
[6475]	valid_0's multi_logloss: 0.14614
[6476]	valid_0's multi_logloss: 0.146137
[6477]	valid_0's multi_logloss: 0.146136
[6478]	valid_0's multi_logloss: 0.146134
[6479]	valid_0's multi_logloss: 0.146135
[6480]	valid_0's multi_logloss: 0.146136
[6481]	valid_0's multi_logloss: 0.146137
[6482]	valid_0's multi_logloss: 0.146136
[6483]	valid_0's multi_logloss: 0.146138
[6484]	valid_0's multi_logloss: 0.146133
[6485]	valid_0's multi_logloss: 0.146132
[6486]	valid_0's multi_logloss: 0.146134
[6487]	valid_0's multi_logloss: 0.146136
[6488]	valid_0's multi_logloss: 0.146137
[6489]	valid_0's multi_logloss: 0.146134
[6490]	valid_0's multi_logloss: 0.146132
[6491]	valid_0's multi_logloss: 0.146132
[6492]	valid_0's multi_logloss: 0.146132
[6493]	valid_0's multi_logloss: 0.146132
[6494]	valid_0's multi_logloss: 0.146133
[6495]	valid_0's multi_logloss: 0.146129
[6496]	valid_0's 

[6673]	valid_0's multi_logloss: 0.145943
[6674]	valid_0's multi_logloss: 0.145947
[6675]	valid_0's multi_logloss: 0.145951
[6676]	valid_0's multi_logloss: 0.145949
[6677]	valid_0's multi_logloss: 0.145948
[6678]	valid_0's multi_logloss: 0.145942
[6679]	valid_0's multi_logloss: 0.14594
[6680]	valid_0's multi_logloss: 0.14594
[6681]	valid_0's multi_logloss: 0.14594
[6682]	valid_0's multi_logloss: 0.145942
[6683]	valid_0's multi_logloss: 0.145943
[6684]	valid_0's multi_logloss: 0.145944
[6685]	valid_0's multi_logloss: 0.145946
[6686]	valid_0's multi_logloss: 0.145947
[6687]	valid_0's multi_logloss: 0.145945
[6688]	valid_0's multi_logloss: 0.145947
[6689]	valid_0's multi_logloss: 0.145947
[6690]	valid_0's multi_logloss: 0.145946
[6691]	valid_0's multi_logloss: 0.145948
[6692]	valid_0's multi_logloss: 0.145949
[6693]	valid_0's multi_logloss: 0.145947
[6694]	valid_0's multi_logloss: 0.145946
[6695]	valid_0's multi_logloss: 0.145949
[6696]	valid_0's multi_logloss: 0.145946
[6697]	valid_0's mu

[6875]	valid_0's multi_logloss: 0.145841
[6876]	valid_0's multi_logloss: 0.145841
[6877]	valid_0's multi_logloss: 0.145842
[6878]	valid_0's multi_logloss: 0.14584
[6879]	valid_0's multi_logloss: 0.14584
[6880]	valid_0's multi_logloss: 0.145837
[6881]	valid_0's multi_logloss: 0.145836
[6882]	valid_0's multi_logloss: 0.145833
[6883]	valid_0's multi_logloss: 0.145832
[6884]	valid_0's multi_logloss: 0.14583
[6885]	valid_0's multi_logloss: 0.145825
[6886]	valid_0's multi_logloss: 0.145823
[6887]	valid_0's multi_logloss: 0.145823
[6888]	valid_0's multi_logloss: 0.145822
[6889]	valid_0's multi_logloss: 0.145824
[6890]	valid_0's multi_logloss: 0.145821
[6891]	valid_0's multi_logloss: 0.145822
[6892]	valid_0's multi_logloss: 0.145824
[6893]	valid_0's multi_logloss: 0.145825
[6894]	valid_0's multi_logloss: 0.145824
[6895]	valid_0's multi_logloss: 0.145822
[6896]	valid_0's multi_logloss: 0.145822
[6897]	valid_0's multi_logloss: 0.145821
[6898]	valid_0's multi_logloss: 0.14582
[6899]	valid_0's mul

[7077]	valid_0's multi_logloss: 0.145664
[7078]	valid_0's multi_logloss: 0.145661
[7079]	valid_0's multi_logloss: 0.145657
[7080]	valid_0's multi_logloss: 0.145657
[7081]	valid_0's multi_logloss: 0.145657
[7082]	valid_0's multi_logloss: 0.145658
[7083]	valid_0's multi_logloss: 0.145659
[7084]	valid_0's multi_logloss: 0.145656
[7085]	valid_0's multi_logloss: 0.145657
[7086]	valid_0's multi_logloss: 0.145657
[7087]	valid_0's multi_logloss: 0.145659
[7088]	valid_0's multi_logloss: 0.145659
[7089]	valid_0's multi_logloss: 0.145659
[7090]	valid_0's multi_logloss: 0.145658
[7091]	valid_0's multi_logloss: 0.145656
[7092]	valid_0's multi_logloss: 0.145658
[7093]	valid_0's multi_logloss: 0.145658
[7094]	valid_0's multi_logloss: 0.145661
[7095]	valid_0's multi_logloss: 0.145661
[7096]	valid_0's multi_logloss: 0.145659
[7097]	valid_0's multi_logloss: 0.145662
[7098]	valid_0's multi_logloss: 0.14566
[7099]	valid_0's multi_logloss: 0.145663
[7100]	valid_0's multi_logloss: 0.145661
[7101]	valid_0's 

[7279]	valid_0's multi_logloss: 0.145549
[7280]	valid_0's multi_logloss: 0.145546
[7281]	valid_0's multi_logloss: 0.145546
[7282]	valid_0's multi_logloss: 0.145545
[7283]	valid_0's multi_logloss: 0.145545
[7284]	valid_0's multi_logloss: 0.145544
[7285]	valid_0's multi_logloss: 0.145543
[7286]	valid_0's multi_logloss: 0.145541
[7287]	valid_0's multi_logloss: 0.145541
[7288]	valid_0's multi_logloss: 0.145543
[7289]	valid_0's multi_logloss: 0.145542
[7290]	valid_0's multi_logloss: 0.145541
[7291]	valid_0's multi_logloss: 0.145542
[7292]	valid_0's multi_logloss: 0.145539
[7293]	valid_0's multi_logloss: 0.145536
[7294]	valid_0's multi_logloss: 0.14554
[7295]	valid_0's multi_logloss: 0.145538
[7296]	valid_0's multi_logloss: 0.145537
[7297]	valid_0's multi_logloss: 0.145538
[7298]	valid_0's multi_logloss: 0.145535
[7299]	valid_0's multi_logloss: 0.145537
[7300]	valid_0's multi_logloss: 0.145539
[7301]	valid_0's multi_logloss: 0.145541
[7302]	valid_0's multi_logloss: 0.14554
[7303]	valid_0's m

In [47]:
print(f'{accuracy_score(y_val, p_val) * 100:.4f}%')
p_tst = clf.predict(test_sel)

93.8519%


In [48]:
#submission
Submission_set = pd.read_csv(sample_file, index_col=0)

In [49]:
Submission_set['class'] = p_tst
print(Submission_set.shape)
Submission_set.head()

(80000, 1)


,class
id,
320000,2.0
320001,0.0
320002,2.0
320003,0.0
320004,2.0


In [50]:
#class 값 카운팅
Submission_set[target_col].value_counts()

2.0    40580
0.0    29998
1.0     9422
Name: class, dtype: int64

In [51]:
#저장하기
Submission_set.to_csv(submission_file)
